In [1]:
#impoorting libraries

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag, pos_tag_sents
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.model_selection import GridSearchCV


In [2]:
#importing training file

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
def remo(features):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",features[0]).split())

def spell(features):
    x=TextBlob(features)
    return x.correct()

In [4]:
train['tweet']=train[['tweet']].apply(remo, axis=1)
test['tweet']=test[['tweet']].apply(remo, axis=1)
    
print(train.head())
print(test.head())

   id  label                                              tweet
0   1      0  when a father is dysfunctional and is so selfi...
1   2      0  thanks for lyft credit i can t use cause they ...
2   3      0                                bihday your majesty
3   4      0      model i love u take with u all the time in ur
4   5      0                  factsguide society now motivation
      id                                              tweet
0  31963  studiolife aislife requires passion dedication...
1  31964  white supremacists want everyone to see the ne...
2  31965  safe ways to heal your acne altwaystoheal heal...
3  31966  is the hp and the cursed child book up for res...
4  31967  3rd bihday to my amazing hilarious nephew eli ...


In [5]:
#stopwords & dataa preprocessing
wnl = WordNetLemmatizer()

#lemmatization
#train['tweet']=train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#test['tweet']=test['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [6]:
#vectorize=TfidfVectorizer(sublinear_tf=True, stop_words='english')
#tfid=vectorize.fit_transform(train['tweet'])
#tfid.shape

In [7]:
#from sklearn.naive_bayes import MultinomialNB
#MNB=MultinomialNB()
#clf = MNB.fit(tfid, train['label'])
#predicted=MNB.predict(tfid)
#np.mean(predicted == train.label)

In [8]:
params = {"tfidf__ngram_range": [(1, 2)], "svc__C": [0.001,.01, .1, 1, 10, 100]}

clf = Pipeline([("tfidf", TfidfVectorizer(sublinear_tf=True, stop_words='english')), ("svc", LinearSVC())])

gs = GridSearchCV(clf, params, cv=10, verbose=2, n_jobs=-1)
gs.fit(train['tweet'], train['label'])
print(gs.best_estimator_)
print(gs.best_score_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.6min finished


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])
0.968775420812


In [9]:

#params = {"tfidf__ngram_range": [(1, 2)], "LR__C": [.01, .1, 1, 10, 100]}

#clf = Pipeline([("tfidf", TfidfVectorizer(sublinear_tf=True, stop_words='english')), ("LR", LogisticRegression())])

#gs = GridSearchCV(clf, params, cv=10, verbose=2, n_jobs=-1)
#gs.fit(train['tweet'], train['label'])
#print(gs.best_estimator_)
#print(gs.best_score_)

In [10]:
predicted = gs.predict(test['tweet'])

In [11]:
#t=vectorize.transform(test['tweet'])
#predicted = MNB.predict(t)

In [12]:
predicted.shape

(17197,)

In [13]:
final_predict = pd.DataFrame(predicted,columns=['label'])
result = pd.DataFrame(test['id'],columns=['id'])
result = pd.concat([result,final_predict],axis=1)
result.to_csv('test_predictions.csv', index=False)